In [95]:
import string
from langdetect import detect
import lyricsgenius
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn
from adjustText import adjust_text
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from scrape_genius import get_lyrics_from_dataframe

In [96]:
df_grammys = pd.read_csv('grammy_df.csv')

In [123]:
get_lyrics_from_dataframe(df_grammys, 'yCZEZPMfq_-XZBsxCS2x1UKScfpa48rnKgTGaHwOsiv6DAzdtFUqCZWdrVQakSb5')

Searching for "Nel Blu Dipinto Di Blu (Volare)"...
Done.
Searching for "Catch A Falling Star"...
Done.
Searching for "Witchcraft (Single)"...
Done.
Searching for "The Chipmunk Song"...
Done.
Searching for "Fever (Single)"...
Done.
Searching for "Mack The Knife"...
Done.
Searching for "The Three Bells"...
Done.
Searching for "Like Young (Single)"...
Done.
Searching for "High Hopes (Single)"...
Done.
Searching for "A Fool Such As I (Single)"...
Done.
Searching for "The Theme From A Summer Place"...
Done.
Searching for "Nice 'N Easy (Single)"...
Done.
Searching for "Mack The Knife (Single)"...
Done.
Searching for "Georgia On My Mind (Single)"...
Done.
Searching for "Are You Lonesome Tonight? (Single)"...
Done.
Searching for "Moon River"...
Done.
Searching for "Up A Lazy River"...
Done.
Searching for "The Second Time Around (Single)"...
Done.
Searching for "Take Five"...
Done.
Searching for "Big Bad John (Single)"...
Done.
Searching for "I Left My Heart In San Francisco"...
Done.
Searching

,title,artist,winner,year,lyrics
0,Nel Blu Dipinto Di Blu (Volare),Domenico Modugno,1,1958,"Nel blu dipinto di blu (Volare) Lyrics[Testo di ""Nel blu dipinto di blu (Volare)""]\n\n[Strofa 1]\nPenso che un sogno così non ritorni mai più\nMi dipingevo le mani e la faccia di blu\nPoi d'improvviso venivo dal vento rapito\nE incominciavo a volare nel cielo infinito\n\n[Ritornello]\nVolare, oh oh\nCantare, oh oh oh oh\nNel blu dipinto di blu\nFelice di stare lassù\n\n[Strofa 2]\nE volavo, volavo felice\nPiù in alto del sole ed ancora più su\nMentre il mondo pian piano spariva, lontano laggiù\nUna musica dolce suonava soltanto per me\n\n[Ritornello]\nVolare, oh oh\nCantare, oh oh oh oh\nNel blu dipinto di blu\nFelice di stare lassù\nYou might also like[Strofa 3]\nMa tutti i sogni nell'alba svaniscon perché\nQuando tramonta la luna li porta con sé\nMa io continuo a sognare negli occhi tuoi belli\nChe sono blu come un cielo trapunto di stelle\n\n[Ritornello]\nVolare, oh oh\nCantare, oh oh oh oh\nNel blu degli occhi tuoi blu\nFelice di stare quaggiù\n\n[Strofa 4]\nE continuo a volare felice\nPiù in alto del sole ed ancora più su\nMentre il mondo pian piano scompare negli occhi tuoi blu\nLa tua voce è una musica dolce che suona per me\n\n[Ritornello]\nVolare, oh oh\nCantare, oh oh oh oh\nNel blu degli occhi tuoi blu\nFelice di stare quaggiù\nNel blu degli occhi tuoi blu\nFelice di stare quaggiù con te1Embed"
1,Catch A Falling Star,Perry Como,0,1958,"Catch a Falling Star Lyrics[Chorus]\nCatch a falling star and put it in your pocket\nNever let it fade away\nCatch a falling star and put it in your pocket\nSave it for a rainy day\n\n[Verse 1]\nFor love may come and tap you on the shoulder some starless night\nJust in case you feel you want to hold her\nYou'll have a pocketful of starlight\n\n[Chorus]\nCatch a falling star and put it in your pocket\nNever let it fade away\nCatch a falling star and put it in your pocket\nSave it for a rainy day\n\n[Verse 1]\nFor love may come and tap you on the shoulder some starless night\nJust in case you feel you want to hold her\nYou'll have a pocketful of starlight\n(Pocketful of starlight, hm, hm, hm, hm, hm, hm)\n\n[Chorus]\nCatch a falling star and put it in your pocket\nNever let it fade away\nCatch a falling star and put it in your pocket\nSave it for a rainy day\nYou might also like[Hook]\n(Save it for a rainy, save it for a rainy, rainy, rainy, day)\n\n[Bridge]\nFor when your troubles start multiplying and they just might\nIt's easy to forget them without trying\nWith just a pocketful of starlight\n\n[Chorus]\nCatch a falling star and put it in your pocket\nNever let it fade away\nCatch a falling star and put it in your pocket\nSave it for a rainy dayEmbed"
2,Witchcraft (Single),Frank Sinatra,0,1958,"Sweetest Maleficia Lyrics[Gilles de Rais:]\nI conjure you\nBarron, Satan, Beelzebub\nBy the Father, the Son, and the Holy Spirit\nBy the Virgin Mary and all the saints\nTo appear, in person\nSo that you may speak to us\nAnd fulfill our desires\n\nCome at my bidding\nAnd I will grant you\nWhatever you want, however vile\nAnd the containing of my life\n\nHe would rise triumphant\nAll done up\nOn a plume of craven wings\nTrafficking with sycophants\nSharing his cup\nAmidst other graver things\n\nAlchemists and sorcerers stitched his head\nWith the stench of pitch and myrrh\n\nThe devout faded out but the pagan remained\nThe candles burnt low and still nothing came\nBearing golden secrets from a cold malevolent race\nYou might also likeHe would have his demon\nHe would have his vice\nAll save his soul was up for sacrifice\nDespite their raising not a single hair\nEverything stank of witchcraft there\n\nFrom the stained chapel to the statued lawn\nIn Caprineum on the lake\nTo the still lit crypts and the slit of dawn\nSliding down the towers, it all smelt fake\n\nHe needed answers not advice\nIntending to devise\nA lengthy train of torture for the fool\nWho thought a seance would suffice\nOr sighted, furre

In [142]:
merged_df = pd.read_csv('genius_grammy.csv')

In [143]:
merged_df.dropna(inplace=True)

In [144]:
merged_df.title.nunique()

343

In [145]:
merged_df

,title,artist,winner,year,lyrics
0,Nel Blu Dipinto Di Blu (Volare),Domenico Modugno,1,1958,"Nel blu dipinto di blu (Volare) Lyrics[Testo di ""Nel blu dipinto di blu (Volare)""]\n\n[Strofa 1]\nPenso che un sogno così non ritorni mai più\nMi dipingevo le mani e la faccia di blu\nPoi d'improvviso venivo dal vento rapito\nE incominciavo a volare nel cielo infinito\n\n[Ritornello]\nVolare, oh oh\nCantare, oh oh oh oh\nNel blu dipinto di blu\nFelice di stare lassù\n\n[Strofa 2]\nE volavo, volavo felice\nPiù in alto del sole ed ancora più su\nMentre il mondo pian piano spariva, lontano laggiù\nUna musica dolce suonava soltanto per me\n\n[Ritornello]\nVolare, oh oh\nCantare, oh oh oh oh\nNel blu dipinto di blu\nFelice di stare lassù\nYou might also like[Strofa 3]\nMa tutti i sogni nell'alba svaniscon perché\nQuando tramonta la luna li porta con sé\nMa io continuo a sognare negli occhi tuoi belli\nChe sono blu come un cielo trapunto di stelle\n\n[Ritornello]\nVolare, oh oh\nCantare, oh oh oh oh\nNel blu degli occhi tuoi blu\nFelice di stare quaggiù\n\n[Strofa 4]\nE continuo a volare felice\nPiù in alto del sole ed ancora più su\nMentre il mondo pian piano scompare negli occhi tuoi blu\nLa tua voce è una musica dolce che suona per me\n\n[Ritornello]\nVolare, oh oh\nCantare, oh oh oh oh\nNel blu degli occhi tuoi blu\nFelice di stare quaggiù\nNel blu degli occhi tuoi blu\nFelice di stare quaggiù con te1Embed"
1,Catch A Falling Star,Perry Como,0,1958,"Catch a Falling Star Lyrics[Chorus]\nCatch a falling star and put it in your pocket\nNever let it fade away\nCatch a falling star and put it in your pocket\nSave it for a rainy day\n\n[Verse 1]\nFor love may come and tap you on the shoulder some starless night\nJust in case you feel you want to hold her\nYou'll have a pocketful of starlight\n\n[Chorus]\nCatch a falling star and put it in your pocket\nNever let it fade away\nCatch a falling star and put it in your pocket\nSave it for a rainy day\n\n[Verse 1]\nFor love may come and tap you on the shoulder some starless night\nJust in case you feel you want to hold her\nYou'll have a pocketful of starlight\n(Pocketful of starlight, hm, hm, hm, hm, hm, hm)\n\n[Chorus]\nCatch a falling star and put it in your pocket\nNever let it fade away\nCatch a falling star and put it in your pocket\nSave it for a rainy day\nYou might also like[Hook]\n(Save it for a rainy, save it for a rainy, rainy, rainy, day)\n\n[Bridge]\nFor when your troubles start multiplying and they just might\nIt's easy to forget them without trying\nWith just a pocketful of starlight\n\n[Chorus]\nCatch a falling star and put it in your pocket\nNever let it fade away\nCatch a falling star and put it in your pocket\nSave it for a rainy dayEmbed"
2,Witchcraft (Single),Frank Sinatra,0,1958,"Sweetest Maleficia Lyrics[Gilles de Rais:]\nI conjure you\nBarron, Satan, Beelzebub\nBy the Father, the Son, and the Holy Spirit\nBy the Virgin Mary and all the saints\nTo appear, in person\nSo that you may speak to us\nAnd fulfill our desires\n\nCome at my bidding\nAnd I will grant you\nWhatever you want, however vile\nAnd the containing of my life\n\nHe would rise triumphant\nAll done up\nOn a plume of craven wings\nTrafficking with sycophants\nSharing his cup\nAmidst other graver things\n\nAlchemists and sorcerers stitched his head\nWith the stench of pitch and myrrh\n\nThe devout faded out but the pagan remained\nThe candles burnt low and still nothing came\nBearing golden secrets from a cold malevolent race\nYou might also likeHe would have his demon\nHe would have his vice\nAll save his soul was up for sacrifice\nDespite their raising not a single hair\nEverything stank of witchcraft there\n\nFrom the stained chapel to the statued lawn\nIn Caprineum on the lake\nTo the still lit crypts and the slit of dawn\nSliding down the towers, it all smelt fake\n\nHe needed answers not advice\nIntending to devise\nA lengthy train of torture for the fool\nWho thought a seance would suffice\nOr sighted, furre

In [146]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocess_text(text):
    # Tokenize the text into individual words
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

    # Stem the words (optional)
    porter = nltk.PorterStemmer()
    stemmed_tokens = [porter.stem(token) for token in filtered_tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(stemmed_tokens)
    return preprocessed_text

merged_df['preprocessed_lyrics'] = merged_df['lyrics'].apply(preprocess_text)


In [147]:
# Identify language of each lyric using langdetect
merged_df['language'] = merged_df['preprocessed_lyrics'].apply(detect)

# Filter out non-English lyrics
df_english = merged_df[merged_df['language'] == 'en']


In [148]:
df_english.dropna(axis=1, inplace=True)

if df_english['preprocessed_lyrics'].dtype != 'object':
    df_english['preprocessed_lyrics'] = df_english['preprocessed_lyrics'].astype(str)

/var/folders/hj/n94x31sj08bdmgwjs6ybxbf40000gn/T/ipykernel_93901/2351279597.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english.dropna(axis=1, inplace=True)


In [149]:
df_english

,title,artist,winner,year,lyrics,preprocessed_lyrics,language
1,Catch A Falling Star,Perry Como,0,1958,"Catch a Falling Star Lyrics[Chorus]\nCatch a falling star and put it in your pocket\nNever let it fade away\nCatch a falling star and put it in your pocket\nSave it for a rainy day\n\n[Verse 1]\nFor love may come and tap you on the shoulder some starless night\nJust in case you feel you want to hold her\nYou'll have a pocketful of starlight\n\n[Chorus]\nCatch a falling star and put it in your pocket\nNever let it fade away\nCatch a falling star and put it in your pocket\nSave it for a rainy day\n\n[Verse 1]\nFor love may come and tap you on the shoulder some starless night\nJust in case you feel you want to hold her\nYou'll have a pocketful of starlight\n(Pocketful of starlight, hm, hm, hm, hm, hm, hm)\n\n[Chorus]\nCatch a falling star and put it in your pocket\nNever let it fade away\nCatch a falling star and put it in your pocket\nSave it for a rainy day\nYou might also like[Hook]\n(Save it for a rainy, save it for a rainy, rainy, rainy, day)\n\n[Bridge]\nFor when your troubles start multiplying and they just might\nIt's easy to forget them without trying\nWith just a pocketful of starlight\n\n[Chorus]\nCatch a falling star and put it in your pocket\nNever let it fade away\nCatch a falling star and put it in your pocket\nSave it for a rainy dayEmbed","catch fall star lyric [ choru ] catch fall star put pocket never let fade away catch fall star put pocket save raini day [ vers 1 ] love may come tap shoulder starless night case feel want hold 'll pocket starlight [ choru ] catch fall star put pocket never let fade away catch fall star put pocket save raini day [ vers 1 ] love may come tap shoulder starless night case feel want hold 'll pocket starlight ( pocket starlight , hm , hm , hm , hm , hm , hm ) [ choru ] catch fall star put pocket never let fade away catch fall star put pocket save raini day might also like [ hook ] ( save raini , save raini , raini , raini , day ) [ bridg ] troubl start multipli might 's easi forget without tri pocket starlight [ choru ] catch fall star put pocket never let fade away catch fall star put pocket save raini dayemb",en
2,Witchcraft (Single),Frank Sinatra,0,1958,"Sweetest Maleficia Lyrics[Gilles de Rais:]\nI conjure you\nBarron, Satan, Beelzebub\nBy the Father, the Son, and the Holy Spirit\nBy the Virgin Mary and all the saints\nTo appear, in person\nSo that you may speak to us\nAnd fulfill our desires\n\nCome at my bidding\nAnd I will grant you\nWhatever you want, however vile\nAnd the containing of my life\n\nHe would rise triumphant\nAll done up\nOn a plume of craven wings\nTrafficking with sycophants\nSharing his cup\nAmidst other graver things\n\nAlchemists and sorcerers stitched his head\nWith the stench of pitch and myrrh\n\nThe devout faded out but the pagan remained\nThe candles burnt low and still nothing came\nBearing golden secrets from a cold malevolent race\nYou might also likeHe would have his demon\nHe would have his vice\nAll save his soul was up for sacrifice\nDespite their raising not a single hair\nEverything stank of witchcraft there\n\nFrom the stained chapel to the statued lawn\nIn Caprineum on the lake\nTo the still lit crypts and the slit of dawn\nSliding down the towers, it all smelt fake\n\nHe needed answers not advice\nIntending to devise\nA lengthy train of torture for the fool\nWho thought a seance would suffice\nOr sighted, furred in dragonflies\nThe signature of Satan on a wall\n\nSweetest Maleficia\n\nAlchemists and sorcerers stitched his head\nWith the stench of pitch and myrrh\n\nPlanchette to Blanchet, from ghosts to a priest\nReturning with a spider for the poisonous feast\nThe Italian astrologer Prelati, spinning sin\nHis fingertips were scented with\nThe tears from seraphim cheeks\nPart glamour and a hammer\nCadaverous and glib\nCommanding in a voice of frozen peaks\n\nHe would have his demon\nHe would have his gold\nOut of control Gilles' soul was

In [150]:
def read_lexicon(filename):
    word_sentiments = {}

    # Parse lexicon file
    with open(filename, 'r') as f:
        for line in f:
            # Ignore blank lines
            if line.strip():
                word, sentiment, value = line.split()
                if value == '1':
                    # Add sentiment to dictionary
                    cur_list = word_sentiments.get(word, [])
                    cur_list.append(sentiment)
                    word_sentiments[word] = cur_list

    return word_sentiments

In [153]:
def get_sentiment_percentages(track_df, sentiment):
    sentiment_pcts = []

    # Parse tracks
    for title in track_df.index.values:
        num_sentiment_words = 0
        all_words = str(track_df.loc[title]['preprocessed_lyrics']).split()

        # Parse words in lyrics
        for word in all_words:
            if sentiment in word_sentiments_dict.get(word, []):
                num_sentiment_words += 1

        sentiment_pcts.append(num_sentiment_words/len(all_words))

    return pd.DataFrame(data=sentiment_pcts, index=track_df.index.values, columns=[sentiment + '_pct'])


In [154]:
#Initialize sentiment dictionary and track dataframe
word_sentiments_dict = read_lexicon('lexicon.txt')


#Generate sentiment percentages based on NRC Lexicon sentiments
sentiments = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust']
for sentiment  in sentiments:
    sentiment_df = get_sentiment_percentages(df_english, sentiment)
    df_english = df_english.join(sentiment_df)

In [155]:
# Calculate the weighted average of the sentiment percentages for each row in the DataFrame
weights = [0.1, 0.4, 0.2, 0.2]
sentiments = ['negative_pct', 'sadness_pct', 'disgust_pct', 'fear_pct']
df_english['gloom_index'] = (df_english[sentiments] * weights).sum(axis=1)

# Rescale the 'gloom_index' column to a range of 0 to 100
max_gloom_value = df_english['negative_index'].max()
min_gloom_value = df_english['negative_index'].min()
df_english['negative_index'] = (df_english['negative_index'] - min_gloom_value) / (max_gloom_value - min_gloom_value) * 100

# Sort the DataFrame by the 'gloom_index' column in descending order
df_english.sort_values(by='negative_index', ascending=False, inplace=True)



In [156]:
df_english

title  \
311                                     Bad Guy   
178     If I Ever Lose My Faith In You (Single)   
241                      Not Ready To Make Nice   
39                    The \"In\" Crowd (Single)   
83                             Help Me (Single)   
..                                          ...   
18                                    Take Five   
98   Love Theme From A Star Is Born (Evergreen)   
36                             A Taste Of Honey   
81                          I Honestly Love You   
50                 By The Time I Get To Phoenix   

                                artist  winner  year  \
311                      Billie Eilish       1  2019   
178                              Sting       0  1993   
241                       Dixie Chicks       1  2006   
39                   Ramsey Lewis Trio       0  1965   
83                       Joni Mitchell       0  1974   
..                                 ...     ...   ...   
18                        Dave Brubeck       0  1961   
98                    Barbra Streisand       0  1977   
36   Herb Alpert And The Tijuana Brass       1  1965   
81                  Olivia Newton-John       1  1974   
50                       Glen Campbell       0  1967   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [157]:
df_english.head(20)

title                    artist  \
311                                  Bad Guy             Billie Eilish   
178  If I Ever Lose My Faith In You (Single)                     Sting   
241                   Not Ready To Make Nice              Dixie Chicks   
39                 The \"In\" Crowd (Single)         Ramsey Lewis Trio   
83                          Help Me (Single)             Joni Mitchell   
2                        Witchcraft (Single)             Frank Sinatra   
32                           People (Single)          Barbra Streisand   
134     Girls Just Want To Have Fun (Single)              Cyndi Lauper   
275         Stronger (What Doesn't Kill You)            Kelly Clarkson   
252                             Paper Planes                     M.I.A   
87                Rhinestone Cowboy (Single)             Glen Campbell   
268                                 The Cave            Mumford & Sons   
33                  I Want To Hold Your Hand               The Beatles   
60                           A Boy Named Sue               Johnny Cash   
66                             It's Too Late               Carole King   
264                     Love The Way You Lie  Eminem Featuring Rihanna   
91                           This Masquerade             George Benson   
326                                   Colors               Black Pumas   
263                                 Fuck You              Cee Lo Green   
191                         Change The World              Eric Clapton   

     winner  year  \
311       1  2019   
178       0  1993   
241       1  2006   
39        0  1965   
83        0  1974   
2         0  1958   
32        0  1964   
134       0  1984   
275       0  2012   
252       0  2008   
87        0  1975   
268       0  2011   
33        0  1964   
60        0  1969   
66        1  1971   
264       0  2010   
91        1  1976   
326       0  2020   
263       0  2010   
191       1  1996   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [139]:
df_english.drop_duplicates(['title','artist', 'year'], inplace=True)

In [140]:
df_english.sort_values(['gloom_index'], ascending=False).head(20)

,title,artist,winner,year,lyrics,language,clean_lyrics,anger_pct,anticipation_pct,disgust_pct,fear_pct,joy_pct,negative_pct,positive_pct,sadness_pct,surprise_pct,trust_pct,gloom_index
39,"The \""In\"" Crowd (Single)",Ramsey Lewis Trio,0,1965,"TranslationsPortuguêsWouldn’t Be a Lie LyricsMaybe I could have changed your mind\nMaybe you could thought about someone else\nRather than yourself, oh anybody but yourself\n\nAnd I know you felt lonely\nYeah I know people try to get you down\nAnd all around those laughing faces\nSingle you out in the crowd\nOh single you out in the crowd\n\nI tried so hard, to help you understand\nTo put some hope into your hands\nAnd all the time, you cried and said to me\n""Why can't everything be right?""\nEach time I have to put up a fight\nAnd the world keeps on winning\nOh yeah the world keeps on winning\n\nWouldn't be a lie if I said I was missing you tonight\nWouldn't be a lie if I said I was missing you\nWouldn't be a lie if I said I was missing you tonight\nWouldn't be a lie if I said I was missing you\nMissing you, missing you\n\nNever thought that you, that you wouldn't be here\nNever thought that I would see these tears fall\nAnd all these lonely times I'm feeling without you\nKnowing I don't have you through it all\nKnowing I don't have you through it all\nYou might also likeNo it wouldn't be a lie if I said I was missing you tonight\nWouldn't be a lie if I said I was missing you\nWouldn't be a lie if I said I was missing you tonight\nWouldn't be a lie if I said I was missing you at all\nWouldn't be a lie if I said I was missing you tonight\nWouldn't be a lie if I said I was missing you\nOh wouldn't be a lie if I said I was missing you tonight\nWouldn't be a lie if I said I was missing you\nMissing you, missing you, missing you\nMissing youEmbed",en,None,0.051282,0.025641,0.054487,0.060897,0.016026,0.102564,0.016026,0.105769,0.012821,0.012821,100.000000
311,Bad Guy,Billie Eilish,1,2019,"TranslationsHrvatskiTürkçeSvenskaEspañolРусскийRomânăPortuguêsPolskiItalianoHebrewDeutschFrançaisفارسیNederlandsDanskChinese TraditionalΕλληνικά​bad guy Lyrics[Verse 1]\nWhite shirt now red, my bloody nose\nSleepin', you're on your tippy toes\nCreepin' around like no one knows\nThink you're so criminal\nBruises on both my knees for you\nDon't say thank you or please\nI do what I want when I'm wanting to\nMy soul? So cynical\n\n[Chorus]\nSo you're a tough guy\nLike it really rough guy\nJust can't get enough guy\nChest always so puffed guy\nI'm that bad type\nMake your mama sad type\nMake your girlfriend mad tight\nMight seduce your dad type\nI'm the bad guy\nDuh\n\n[Post-Chorus]\nI'm the bad guy\n\n[Verse 2]\nI like it when you take control\nEven if you know that you don't\nOwn me, I'll let you play the role\nI'll be your animal\nMy mommy likes to sing along with me\nBut she won't sing this song\nIf she reads all the lyrics\nShe'll pity the men I know\nYou might also like[Chorus]\nSo you're a tough guy\nLike it really rough guy\nJust can't get enough guy\nChest always so puffed guy\nI'm that bad type\nMake your mama sad type\nMake your girlfriend mad tight\nMight seduce your dad type\nI'm the bad guy\nDuh\n\n[Post-Chorus]\nI'm the bad guy, duh\nI'm only good at bein' bad, bad\n\n[Bridge]\nI like when you get mad\nI guess I'm pretty glad that you're alone\nYou said she's scared of me?\nI mean, I don't see what she sees\nBut maybe it's 'cause I'm wearing your cologne\n\n[Outro]\nI'm a bad guy\nI'm, I'm a bad guy\nBad guy, bad guy\nI'm a bad345Embed",en,None,0.054945,0.018315,0.058608,0.054945,0.018315,0.073260,0.018315,0.073260,0.007326,0.014652,78.450363
275,Stronger (What Doesn't Kill You),Kelly Clarkson,0,2012,"TranslationsΕλληνικάStronger (What Doesn’t Kill You) Lyrics[Verse 1]\nYou know the bed feels warmer\nSleepin' here alone (Sleepin' here alone)\nYou know I dream in color\nAnd do the things I want (Do the things I want)\n\n[Pre-Chorus]\nYou think you got the best of me\nThink you've had the last laugh\